In [1]:
import os, glob
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # ignore info and warnings
import pandas as pd

from utils import DatasetWrapper, ModelWrapper

/home/nic/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.5.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [2]:
dataset = DatasetWrapper(dataset_path='/data/train', img_size=224, batch_size=64)
dataset.make_datagens(augment=True, hsv_mask=True)
model = ModelWrapper(dataset)
model.make_model(figname='hsv_augmented')
model.model.load_weights(os.path.join(model.weightdir, model.figname + '.h5'))

Found 3330 images belonging to 12 classes.
Found 1420 images belonging to 12 classes.


In [3]:
paths = sorted(glob.glob('/data/test/*.png'))

In [5]:
import tensorflow.keras.preprocessing as tfkp
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from utils import get_hsv_masked
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import cv2

In [6]:
df = dict(file=[], species=[])
mapping = {v: k for k, v in dataset.train_generator.class_indices.items()}
for p in paths:
    img = np.array(tfkp.image.load_img(p))
    img = tfkp.image.smart_resize(
        img, (224, 224), interpolation='bilinear')
    img = np.array(img)
    img = preprocess_input(get_hsv_masked(img))
    img = tf.expand_dims(img, 0)
    i = model.model(img)
    df['file'].append(p.split('/')[-1])
    df['species'].append(mapping[i.numpy().argmax()])

In [7]:
df = pd.DataFrame.from_dict(df)
df

,file,species
0,0021e90e4.png,Small-flowered Cranesbill
1,003d61042.png,Fat Hen
2,007b3da8b.png,Sugar beet
3,0086a6340.png,Common Chickweed
4,00c47e980.png,Sugar beet
...,...,...
789,fea355851.png,Loose Silky-bent
790,fea3da57c.png,Sugar beet
791,fef2ade8c.png,Sugar beet
792,ff65bc002.png,Charlock


In [11]:
df.to_csv('results/submission.csv', index=False)

~ 596 rank; 0.88413 score